In [10]:
!pip install langchain

In [11]:
!pip install langchain-openai
!pip install langchain_community

In [12]:
!pip install sentence_transformers

In [13]:
!pip install chromadb

In [15]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [16]:
# import libraries
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from google.colab import userdata

In [17]:
!pip install pypdf

In [18]:
CHROMA_PATH = "Chroma"
# ----- Data Indexing Process -----
# load your pdf docs
DOC_PATH = "/content/20220202_alphabet_10K.pdf"
# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()
# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
#embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={'device': 'cpu'})
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embed the chunks as vectors and load them into the database.
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
#query = 'what are the top risks mentioned in the document?'
query='Summarize the alphabet_10K_2022 document'

In [19]:
# retrieve context - top 5 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])
# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""


In [21]:
# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
#using google genai instead of open AI here

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash" ,api_key=userdata.get('GoogleAPIKey'))

response_text = llm.predict(prompt)
print(response_text)

The context provides information for Alphabet Inc.'s Form 10-K for the fiscal year ended December 31, 2021. This document includes a Table of Contents starting with "Note About Forward-Looking Statements" on page 3. PART I of this 10-K covers Item 1. Business (page 4), Item 1A. Risk Factors (page 10), Item 1B. Unresolved Staff Comments (page 24), Item 2. Properties (page 24), Item 3. Legal Proceedings (page 24), and Item 4. Mine Safety Disclosures (page 24). PART II includes Item 5. Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities (page 25) and Item 6. [Reserved] (page 27). The context also mentions "See accompanying notes." and refers to page 50 in relation to the Table of Contents.

Additionally, the context refers to "Alphabet Inc. Amended and Restated 2012 Stock Plan - Form of Alphabet Restricted Stock Unit Agreement" and "Alphabet Inc. Amended and Restated 2012 Stock Plan - Performance Stock Unit Agreement" from an Annual